<a href="https://colab.research.google.com/github/crescent98/machinelearning35911/blob/master/assignment11_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt -y install libcusparse8.0 libnvrtc8.0 libnvtoolsext1
!ln -snf /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so.8.0 /usr/lib/x86_64-linux-gnu/libnvrtc-builtins.so
!pip install cupy-cuda80

Reading package lists... Done
Building dependency tree       
Reading state information... Done
E: Unable to locate package libcusparse8.0
E: Couldn't find any package by glob 'libcusparse8.0'
E: Couldn't find any package by regex 'libcusparse8.0'
E: Unable to locate package libnvrtc8.0
E: Couldn't find any package by glob 'libnvrtc8.0'
E: Couldn't find any package by regex 'libnvrtc8.0'
     |████████████████████████████████| 208.7MB 65kB/s 


In [ ]:

%matplotlib inline
import matplotlib.pyplot as plt
import six
import numpy as np
import cupy as cp

ImportError: ignored

In [1]:
import cupy as cp

In [7]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
import pickle
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


Google Drive Mounted

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
review_data = load_files(r"/content/drive/My Drive/movie_review")
X, y = review_data.data, review_data.target

documents = []

stemmer = WordNetLemmatizer()

for sen in range(0, len(X)):
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(X[sen]))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    
    # Remove single characters from the start
    document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()
    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)

vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X = vectorizer.fit_transform(documents).toarray()

tfidfconverter = TfidfTransformer()
X = tfidfconverter.fit_transform(X).toarray()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)


In [20]:
#
# normalize the values of the input data to be [0, 1]
#
def normalize(data):

    data_normalized = (data - min(data)) / (max(data) - min(data))

    return(data_normalized)

#
# example of distance function between two vectors x and y
#
def distance(x, y):

    d = (x - y) ** 2
    s = cp.sum(d)
    # r = np.sqrt(s)

    return(s)

In [ ]:
normalize(np.arange(10))

array([0.        , 0.11111111, 0.22222222, 0.33333333, 0.44444444,
       0.55555556, 0.66666667, 0.77777778, 0.88888889, 1.        ])

In [ ]:
np.max(X_train[0])

0.27764702945292935

In [23]:
np.shape(X_train)

(1401, 1500)

Normalization

In [25]:
X_train_gpu=cp.array(X_train)
X_test_gpu=cp.array(X_test)

train_x=cp.zeros((1401,1500))
test_x=cp.zeros((601,1500))

for count in range(1401):
  train_x[count]=normalize(X_train_gpu[count])

for count in range(0,601):
  test_x[count]=normalize(X_test_gpu[count])

In [ ]:
# Shape of Data

print(np.shape(X_train))
print(np.shape(X_test))
print(np.shape(y_train))
print(np.shape(y_test))

(1401, 1500)
(601, 1500)
(1401,)
(601,)


In [ ]:
# Shape of Data (by Cupy)

print(cp.ndim(train_x))
print(cp.ndim(test_x))



Functions

In [12]:
# ndarray를 받았을 때 vector 곱과 sigmoid 연산
def nextA(x,theta):
  mul=cp.matmul(theta,x)
  return 1/(1+cp.exp(-mul))

In [13]:
# 4 thetas

theta_1=cp.random.normal(0,0.1,(1300,1501))
theta_2=cp.random.normal(0,0.1,(300,1301))
theta_3=cp.random.normal(0,0.1,(200,301))
theta_4=cp.random.normal(0,0.1,(1,201))
"""
theta_1=np.random.normal(0.1,0,(1000,1501))
theta_2=np.random.normal(0.1,0,(600,1001))
theta_3=np.random.normal(0.1,0,(400,601))
theta_4=np.random.normal(0.1,0,(250,401))
theta_5=np.random.normal(0.1,0,(1,251))
"""

'\ntheta_1=np.random.normal(0.1,0,(1000,1501))\ntheta_2=np.random.normal(0.1,0,(600,1001))\ntheta_3=np.random.normal(0.1,0,(400,601))\ntheta_4=np.random.normal(0.1,0,(250,401))\ntheta_5=np.random.normal(0.1,0,(1,251))\n'

In [14]:
train_objective=cp.zeros(5000)
train_accuracy=cp.zeros(5000)

test_objective=cp.zeros(5000)
test_accuracy=cp.zeros(5000)

theta_size=cp.size(theta_1)+cp.size(theta_2)+cp.size(theta_3)+cp.size(theta_4)

m=1401 #train은 1401회

In [16]:
alpha=0.3
lamb=1000

In [17]:
i=0

Neural Network Algorithm

In [18]:
while(i<100):

  #training
  b_delta5=0
  b_delta4=0
  b_delta3=0
  b_delta2=0

  objective_sum=0

  cor_count=0
  theta_aver=(cp.sum(cp.square(theta_1))+cp.sum(cp.square(theta_2))+cp.sum(cp.square(theta_3))+cp.sum(cp.square(theta_4)))/theta_size
  
  #training
  for j in range(0,1401):

    # Forward Propagation

    a1=train_x[j]
    a1_=cp.concatenate(cp.ones(1),a1)
    a2=nextA(a1_,theta_1)
    a2_=cp.concatenate(cp.ones(1),a2)
    a3=nextA(a2_,theta_2)
    a3_=cp.concatenate(cp.ones(1),a3)
    a4=nextA(a3_,theta_3)
    a4_=cp.concatenate(cp.ones(1),a4)
    a5=nextA(a4_,theta_4)
    a5_=cp.concatenate(cp.ones(1),a5)

    # Compute Delta5
    delta_5=a5-y_train[j]

    # Compute Delta4, Delta3, Delta2
    delta_4=cp.matmul(cp.transpose(theta_4),delta_5)*a4_*(1-a4_)
    delta_3=cp.matmul(cp.transpose(theta_3),delta_4[1:])*a3_*(1-a3_)
    delta_2=cp.matmul(cp.transpose(theta_2),delta_3[1:])*a2_*(1-a2_)

    # Compute(accumulate) big Delta5, Delta4, Delta3, Delta2
    b_delta5=b_delta5+cp.dot(delta_5.reshape(1,1),a4_.reshape(1,201))
    b_delta4=b_delta4+cp.dot(delta_4[1:].reshape(200,1),a3_.reshape(1,301))
    b_delta3=b_delta3+cp.dot(delta_3[1:].reshape(300,1),a2_.reshape(1,1301))
    b_delta2=b_delta2+cp.dot(delta_2[1:].reshape(1300,1),a1_.reshape(1,1501))
  
    # Compute(accumulate) Objective
    # L is based on the Correct Answer(Y)
    objective_sum=objective_sum+cp.sum((-y_train[j]*cp.log(a5))-(1-y_train[j])*cp.log(1-a5))

    # Compute the calculated_answer
    if a5>1/2:
      ans=1
    else:
      ans=0
    

  #Setting new Theta
  theta_4=theta_4-alpha*(b_delta5/1401+lamb*theta_4/theta_size)
  theta_3=theta_3-alpha*(b_delta4/1401+lamb*theta_3/theta_size)
  theta_2=theta_2-alpha*(b_delta3/1401+lamb*theta_2/theta_size)
  theta_1=theta_1-alpha*(b_delta2/1401+lamb*theta_1/theta_size)

  # Testing

  ans2=0
  cor_count2=0
  objective_sum2=0

  for j in range(0,601):

    # Compute a5 for objective (Forward Propagation)

    a1=test_x[j]
    a1_=cp.concatenate(cp.ones(1),a1)
    a2=nextA(a1_,theta_1)
    a2_=cp.concatenate(cp.ones(1),a2)
    a3=nextA(a2_,theta_2)
    a3_=cp.concatenate(cp.ones(1),a3)
    a4=nextA(a3_,theta_3)
    a4_=cp.concatenate(cp.ones(1),a4)
    a5=nextA(a4_,theta_4)
    a5_=cp.concatenate(cp.ones(1),a5)

    # Computing(accumulating) Objective Function
    objective_sum2=objective_sum2+cp.sum((-y_test[j]*cp.log(a5))-(1-y_test[j])*cp.log(1-a5))

    # Compute the answer
    if a5>1/2:
      ans2=1
    else:
      ans2=0
    
    #Compute the accuracy

  # Calcluate the Objective Function
  train_objective[i]=objective_sum/1401+theta_aver*lamb/2
  test_objective[i]=objective_sum2/601+theta_aver*lamb/2

  # Checking Accuracy -> Matrix


  # Next Step
  i=i+1
  

NameError: ignored

In [ ]:
a1=train_x[1]
a1_=np.append(np.ones(1),a1)
a2=nextA(a1_,theta_1)
a2_=np.append(np.ones(1),a2)
a3=nextA(a2_,theta_2)
a3_=np.append(np.ones(1),a3)
a4=nextA(a3_,theta_3)
a4_=np.append(np.ones(1),a4)
a5=nextA(a4_,theta_4)
a5_=np.append(np.ones(1),a5)

In [ ]:
for i in range(0,20):
  a1=train_x[i]
  a1_=np.append(np.ones(1),a1)
  a2=nextA(a1_,theta_1)
  a2_=np.append(np.ones(1),a2)
  a3=nextA(a2_,theta_2)
  a3_=np.append(np.ones(1),a3)
  a4=nextA(a3_,theta_3)
  a4_=np.append(np.ones(1),a4)
  a5=nextA(a4_,theta_4)
  a5_=np.append(np.ones(1),a5)
  print(a5[0],y_test[i])

0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 0
0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 0
0.9999999981349912 0
0.9999999981349912 0
0.9999999981349912 0
0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 0
0.9999999981349912 0
0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 1
0.9999999981349912 0
0.9999999981349912 0


In [ ]:
for i in range(0,20):
  a1=X_train[i]
  a1_=np.append(np.ones(1),a1)
  a2=nextA(a1_,theta_1)
  a2_=np.append(np.ones(1),a2)
  a3=nextA(a2_,theta_2)
  a3_=np.append(np.ones(1),a3)
  a4=nextA(a3_,theta_3)
  a4_=np.append(np.ones(1),a4)
  a5=nextA(a4_,theta_4)
  a5_=np.append(np.ones(1),a5)
  a6=nextA(a5_,theta_5)
  a6_=np.append(np.ones(1),a6)
  print(a6[0])

0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336
0.9999999999874336


Codes for Evaluation

In [30]:
classifier = train_classifier(X_train, y_train)
y_pred_test = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

NameError: ignored

In [29]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_train,y_pred_train))
print(classification_report(y_train,y_pred_train))
print(accuracy_score(y_train, y_pred_train))

print(confusion_matrix(y_test,y_pred_test))
print(classification_report(y_test,y_pred_test))
print(accuracy_score(y_test,y_pred_test))

NameError: ignored

In [ ]:
classifier = train_classifier(X_train, y_train)
y_pred_test = classifier.predict(X_test)
y_pred_train = classifier.predict(X_train)

NameError: ignored